# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = '../WeatherPy/output_data/cities.csv'
city_df = pd.read_csv(file_path)
city_df.head()

,Unnamed: 0,City Name,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,0,Wittmund,53.57,7.78,43.59,93,5.82,75
1,1,Bluff,-46.60,168.33,50.00,93,8.81,48
2,2,Qiongshan,20.01,110.35,62.60,93,8.95,90
3,3,Tura,25.52,90.22,59.02,54,1.72,0
4,4,Rikitea,-23.12,-134.97,77.38,71,9.33,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API Key
gmaps.configure(api_key=g_key)

In [4]:
#Store Latitude and Longitude into Locations
locations = city_df[["Latitude", "Longitude"]].astype(float)
humid = city_df["Humidity"]

In [5]:
#Add Heatmap layer to map
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Create New Vacation DataFrame
vacation_df = city_df.loc[(city_df["Temperature"] > 70) & (city_df["Temperature"] < 80) & (city_df["Wind Speed"] < 10) & (city_df["Cloudiness"] == 0)].dropna()
vacation_df.head()

,Unnamed: 0,City Name,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
4,4,Rikitea,-23.12,-134.97,77.38,71,9.33,0
15,15,Salalah,17.02,54.09,75.20,69,2.24,0
54,54,Saint-Pierre,-21.34,55.48,72.32,83,2.24,0
68,68,Pisco,-13.70,-76.22,70.75,77,9.17,0
88,88,Arlit,18.74,7.39,73.96,18,4.85,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create Hotel Dataframe from Vacation Dataframe
hotel_df = vacation_df.loc[:,["City Name", "Latitude", "Longitude"]]
hotel_df.head()

,City Name,Latitude,Longitude
4,Rikitea,-23.12,-134.97
15,Salalah,17.02,54.09
54,Saint-Pierre,-21.34,55.48
68,Pisco,-13.70,-76.22
88,Arlit,18.74,7.39


In [10]:
#Add Hotel Name column to Hotel Dataframe
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City Name,Latitude,Longitude,Hotel Name
4,Rikitea,-23.12,-134.97,
15,Salalah,17.02,54.09,
54,Saint-Pierre,-21.34,55.48,
68,Pisco,-13.70,-76.22,
88,Arlit,18.74,7.39,


In [15]:
#Find Hotel Name Based on Latitude and Longitude

#Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Params variables
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#Iterate through hotel dataframe to store lat, lon, city information
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    city_name = row["City Name"]
    
    # Store location inside parems dictionary
    params["location"] = f"{lat},{lon}"

    # Create URL and Make Request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #Get Results
    results = response['results']
    
    # Input hotel name in dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if no hotel, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 4: Rikitea.
Closest hotel in Rikitea is Pension Bianca & Benoit.
------------
Retrieving Results for Index 15: Salalah.
Closest hotel in Salalah is Al Jabal.
------------
Retrieving Results for Index 54: Saint-Pierre.
Closest hotel in Saint-Pierre is Le Battant Des Lames.
------------
Retrieving Results for Index 68: Pisco.
Closest hotel in Pisco is Hotel Presidente.
------------
Retrieving Results for Index 88: Arlit.
Closest hotel in Arlit is Hôtel Telwa Bungalow.
------------
Retrieving Results for Index 90: Alappuzha.
Closest hotel in Alappuzha is Palmgrove Lake Resort.
------------
Retrieving Results for Index 153: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 181: Saint-Joseph.
Closest hotel in Saint-Joseph is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 211: Tessalit.
No hotel
------------
Retrieving Results for Index 213: Hambantota.
Closest hotel in Ham

In [16]:
#Call Hotel Dataframe
hotel_df

,City Name,Latitude,Longitude,Hotel Name
4,Rikitea,-23.12,-134.97,Pension Bianca & Benoit
15,Salalah,17.02,54.09,Al Jabal
54,Saint-Pierre,-21.34,55.48,Le Battant Des Lames
68,Pisco,-13.70,-76.22,Hotel Presidente
88,Arlit,18.74,7.39,Hôtel Telwa Bungalow
90,Alappuzha,9.49,76.33,Palmgrove Lake Resort
153,Saint-Philippe,-21.36,55.77,Les Embruns Du Baril
181,Saint-Joseph,-21.37,55.62,"""Plantation Bed and Breakfast"
211,Tessalit,20.20,1.01,
213,Hambantota,6.12,81.12,The Peacock Beach Hotel Hambantota


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))